In [91]:
import numpy as np
import pandas as pd
from datetime import datetime
import tqdm

checkin_file = "../dataset/test_total.txt"
df = pd.read_csv(checkin_file, sep='\t', header=None)
df.columns = ["user", "poi", "time"]

In [92]:
len(df)

425648

In [93]:
prev_cnt = 0
curr_cnt = len(df)
while prev_cnt != curr_cnt:
    prev_cnt = curr_cnt
    df = df[df.groupby('user').user.transform(len) > 5]
    df = df[df.groupby('poi').poi.transform(len) > 5]
    curr_cnt = len(df)

In [94]:
len(df)

242686

In [95]:
poi2id = np.load("./npy/poi2id.npy").item()
df['poi'] = df['poi'].apply(lambda x: poi2id[x] if poi2id.get(x) != None else 13187)

In [96]:
df['time'] = df['time'].apply(lambda x: (datetime.strptime(x, "%Y-%m-%d %H:%M:%S")-datetime(2009,1,1))\
                              .total_seconds()/360)  # hour

In [97]:
np.max(df.poi)

13005

In [98]:
'''
user2id = {'unk':0}
id2user = [0]
for target_idx in tqdm.tqdm(xrange(len(df))):
    (user, poi, time) = df.iloc[target_idx]
    if user2id.get(user) == None:
        user2id[user] = len(id2user)
        id2user.append(user)
'''
user2id = np.load('./npy/user2id.npy').item()
id2user = np.load('./npy/id2user.npy')

df['user'] = df['user'].apply(lambda x: user2id[x] if user2id.get(x) != None else 0)

In [99]:
len(df.groupby('user')),len(df)

(18712, 242686)

In [100]:
train_user = []
train_context = []
train_target = []

tow = 6
prev_user = df.iloc[0]['user']
user_user = []
user_context = []
user_target = []
for target_idx in tqdm.tqdm(xrange(len(df))):
    (user, poi, time) = df.iloc[target_idx]
    if prev_user != user:
        prev_user = user
        train_user += user_user
        train_context += user_context
        train_target += user_target

        user_user = []
        user_context = []
        user_target = []
        #print train_user, train_context, train_target
    
    context = []
    for context_idx in xrange(target_idx+1, len(df)):
        (c_user, c_poi, c_time) = df.iloc[context_idx]
        if user == c_user and (time+tow) > c_time:
                context.append(c_poi)
        else:
            break
    if context:
        user_user.append(user)
        user_context.append(context)
        user_target.append(poi)
        
train_user += user_user
train_context += user_context
train_target += user_target

100%|██████████| 242686/242686 [01:24<00:00, 2883.45it/s]


In [101]:
len(train_user), len(train_context), len(train_target)

(44189, 44189, 44189)

In [102]:
len_context = []
for i, context in enumerate(train_context):
    len_context.append(len(context))
print np.max(len_context), np.mean(len_context), np.median(len_context), np.min(len_context)

41 2.94349272443 2.0 1


In [103]:
len_context.sort()
print len(len_context)
print len_context[int(len(len_context)*0.99)]

44189
21


In [104]:
maxlen_context = 16
for i, context in enumerate(train_context):
    if len(context) < maxlen_context:
        train_context[i] += ([0]*(maxlen_context-len(context)))
    elif len(context) > maxlen_context:
        train_context[i] = context[:maxlen_context]

In [105]:
len_context = []
for context in train_context:
    len_context.append(len(context))
print np.max(len_context), np.mean(len_context), np.median(len_context), np.min(len_context)

16 16.0 16.0 16


In [106]:
np.save('./npy/test_context.npy', train_context)
np.save('./npy/test_user.npy', train_user)
np.save('./npy/test_target.npy', train_target)

In [63]:
np.save('./npy/user2id.npy', user2id)
np.save('./npy/id2user.npy', id2user)

In [101]:
len(id2user)

4627